In [2]:
import pickle as pkl
import osmnx as ox
import networkx as nx
import heapq
from itertools import count
import graph_utils

In [3]:
graph = pkl.load(open("./data/massachusetts_drive.pkl",'rb'))

In [129]:
length_dict = {}
class StrategyDijkstraWithLimit():
     # Assume Limit is some percentage of the shortest path [0,1]
    def __init__(self, graph, limit):
        self.graph = graph
        self.limit = limit

   
    def get_route(self, source, goal, edge_weight='elevation_change'):
        graph = self.graph

        length_dijkstra = StrategyDijkstra(graph)
        shortest_path = length_dijkstra.get_route(source, goal)
        shortest_path_length = graph_utils.get_path_length(graph, shortest_path)
        print("Shortest Path Length: ", shortest_path_length)

        max_path_length = shortest_path_length + shortest_path_length * self.limit
        print("Maximum Theoretical Length: ", max_path_length)

        node_elevation_weight_func = weight_function(graph, 'elevation_change')
        length_func = weight_function(graph, 'length')

        paths = {source: [source]}

        successor_graph = graph._succ if graph.is_directed() else graph_adj

        push = heapq.heappush
        pop = heapq.heappop
        dist = {} # Dictionary of Final Weights
        # length_dict = {} # Dictionary of Lengths
        seen = {}
        
        c = count()
        queue = []
        if source not in graph:
            return [] # Figure out way to handle exceptions properly
        candidate_paths = []
        seen[source] = 0
        push(queue, (0, next(c), source, 0))
        while queue:
            d, _, v, dist_so_far = pop(queue)
            
            if v in dist:
                continue # already searched this node
            dist[v] = d
            # length_dict[v] = dist_so_far
            if v == 65811502:
                print(paths[v])
            if v == goal:
                candidate_paths.append(paths[v])
                continue
                # break
            for u, e in successor_graph[v].items():
                u_cost = max(0, node_elevation_weight_func(v, u, e))
                u_length = length_func(v, u, e)


                if u_cost is None: continue

                vu_cost = dist[v] + u_cost
                vu_length = dist_so_far + u_length
                # if u in dist:
                #     if vu_cost < dist[u]:
                #         pass # Contradictory paths found, negative weights?
                # elif u not in seen or vu_cost < seen[u]:
                # if (u not in seen or vu_cost < seen[u]) and vu_length <= max_path_length:
                if  vu_length <= max_path_length:
                    seen[u] = vu_cost
                    length_dict[u] = vu_length
                    push(queue, (vu_cost, next(c), u, vu_length))
                    paths[u] = paths[v]+[u]

        return candidate_paths

dijkstra = StrategyDijkstraWithLimit(graph, 0)
paths = dijkstra.get_route(69025899,65777405)
for path in paths:
    print(graph_utils.get_path_length(graph, path))
# length_dict[65777405]

Shortest Path Length:  4975.982
Maximum Theoretical Length:  4975.982
[69025899, 69025899, 69071341, 69071905, 69057871, 69069408, 69068467, 69068236, 69070687, 69022485, 5724225644, 68973495, 69048114, 68997244, 68963642, 5916268622, 69069740, 68980659, 69048969, 68961526, 69028126, 68995324, 69028571, 68982213, 69033033, 68958587, 68959400, 69053365, 69000615, 69005434, 1937532207, 65811502]


Shortest Path Length:  4975.982
Maximum Theoretical Length:  4975.982
4896.566999999999


In [128]:
class StrategyDijkstra():
    def __init__(self, graph):
        self.graph = graph

    def get_route(self, source, goal, edge_weight='length'):
        graph = self.graph
        weight = weight_function(graph, edge_weight)

        paths = {source: [source]}

        successor_graph = graph._succ if graph.is_directed() else graph._adj

        push = heapq.heappush
        pop = heapq.heappop
        dist = {} # Dictionary of Final distances
        seen = {}
        
        c = count()
        queue = []
        if source not in graph:
            return [] # Figure out way to handle exceptions properly
        seen[source] = 0
        push(queue, (0, next(c), source))
        while queue:
            d, _, v = pop(queue)
            if v in dist:
                continue # already searched this node
            dist[v] = d
            if v == goal:
                break
            for u, e in successor_graph[v].items():
                cost = weight(v, u, e)

                if cost is None: continue

                vu_dist = dist[v] + cost
                if u in dist:
                    if vu_dist < dist[u]:
                        pass # Contradictory paths found, negative weights?
                elif u not in seen or vu_dist < seen[u]:
                    seen[u] = vu_dist
                    push(queue, (vu_dist, next(c), u))
                    paths[u] = paths[v]+[u]
        return paths[goal]

def weight_function(graph, weight='length'):
    if weight == 'length':
        def weight_(source, dest, edge_data):
            return min(attr.get(weight, 1) for attr in edge_data.values())
    elif weight =='elevation_change':
        def weight_(source, dest, edge_data):
            return max(0, min(attr.get(weight, 1) for attr in edge_data.values()))
    return weight_

In [154]:
length_dict = {}
class StrategyDijkstraWithLimitMin():
     # Assume Limit is some percentage of the shortest path [0,1]
    def __init__(self, graph, limit):
        self.graph = graph
        self.limit = limit

   
    def get_route(self, source, goal, edge_weight='elevation_change'):
        graph = self.graph

        length_dijkstra = StrategyDijkstra(graph)
        shortest_path = length_dijkstra.get_route(source, goal)
        shortest_path_length = graph_utils.get_path_length(graph, shortest_path)
        print("Shortest Path Length: ", shortest_path_length)
        print ("Shortest Path Elevation Change: ",graph_utils.get_path_elevation(graph, shortest_path))
        max_path_length = shortest_path_length + shortest_path_length * self.limit
        print("Maximum Theoretical Length: ", max_path_length)

        least_elevation = length_dijkstra.get_route(source, goal, edge_weight='elevation_change')
        least_elevation_length = graph_utils.get_path_length(graph, least_elevation)
        print("Elevation Path Length: ", least_elevation_length)
        print("Elevation Path Elevation Change: ", graph_utils.get_path_elevation(graph, least_elevation))

        if least_elevation_length > max_path_length:
            length = len(least_elevation)
            for i in range(2, length):
                node = least_elevation[-i]
                path_length_to_node = graph_utils.get_path_length(graph, least_elevation[:-i+1])
                node_to_goal_shortest = length_dijkstra.get_route(node, goal)
                new_path_length = graph_utils.get_path_length(graph, node_to_goal_shortest)
                if path_length_to_node + new_path_length <= max_path_length:
                    # print(graph_utils.get_path_length(graph, least_elevation[:-i+1] + node_to_goal_shortest))
                    return least_elevation[:-i] + node_to_goal_shortest
        else:
            return least_elevation
                


dijkstra = StrategyDijkstraWithLimit(graph, 0.05)
path = dijkstra.get_route(69025899,65777405)
path_length = graph_utils.get_path_length(graph, path)
print("Hybrid Path Length: ", path_length )
print("Hybrid Path Elevation Change:", graph_utils.get_path_elevation(graph, path))
print(path)

Shortest Path Length:  4975.982
Shortest Path Elevation Change:  54
Maximum Theoretical Length:  5224.7811
Elevation Path Length:  5970.821999999999
Elevation Path Elevation Change:  48
Hybrid Path Length:  5079.494
Hybrid Path Elevation Change: 54
[69025899, 69071341, 69038537, 69011898, 69069408, 68970727, 69029257, 68980206, 69048114, 68997244, 68963642, 5916268622, 69069740, 68980659, 69048969, 68961526, 69028126, 68995324, 69028571, 68982213, 69033033, 68958587, 68959400, 69053365, 69000615, 69005434, 1937532207, 65811502, 65854123, 65875610, 65777405]


In [194]:
length_dict = {}
class StrategyDijkstraWithLimitMax():
     # Assume Limit is some percentage of the shortest path [0,1]
    def __init__(self, graph, limit):
        self.graph = graph
        self.limit = limit

   
    def get_route(self, source, goal, edge_weight='elevation_change'):
        graph = self.graph

        length_dijkstra = StrategyDijkstra(graph)
        shortest_path = length_dijkstra.get_route(source, goal)
        shortest_path_length = graph_utils.get_path_length(graph, shortest_path)
        print("Shortest Path Length: ", shortest_path_length)
        print ("Shortest Path Elevation Change: ",graph_utils.get_path_elevation(graph, shortest_path))
        max_path_length = shortest_path_length * (1+ self.limit)
        print("Maximum Theoretical Length: ", max_path_length)

        # least_elevation = length_dijkstra.get_route(source, goal, edge_weight='elevation_change')
        # least_elevation_length = graph_utils.get_path_length(graph, least_elevation)
        # print("Elevation Path Length: ", least_elevation_length)
        # print("Elevation Path Elevation Change: ", graph_utils.get_path_elevation(graph, least_elevation))
        
        max_path = []
        length_allowance = max_path_length - shortest_path_length
        for i in range(0, len(shortest_path)-1):
            
            cur_node = shortest_path[i]
            next_node = shortest_path[i+1]
            
            highest_elevation = -1
            best_path = []
            for path in nx.all_simple_paths(graph, cur_node, next_node, cutoff=5):
                min_distance = graph[cur_node][next_node][0]['length']
                # if (len(path) > 0): print(cur_node, next_node, path)
                path_elevation = graph_utils.get_path_elevation(graph, path)
                path_length = graph_utils.get_path_length(graph, path)
                if path_elevation > highest_elevation and path_length <= length_allowance + min_distance:
                    highest_elevation = path_elevation
                    best_path = path
            best_path_length = graph_utils.get_path_length(graph, best_path)
            length_allowance -= (best_path_length - min_distance)
            # print("BestPath", cur_node, next_node, best_path)
            for i in best_path[:-1]:
                # print(i)
                max_path.append(i)
            # break
        max_path.append(goal)
        print(max_path)
        return max_path
            


                


dijkstra = StrategyDijkstraWithLimit(graph, 0.5)
path = dijkstra.get_route(69025899,65777405)
# print(path)
path_length = graph_utils.get_path_length(graph, path)
print("Maximum Elevation Path Length: ", path_length )
print("Maximum Elevation Path Elevation Change:", graph_utils.get_path_elevation(graph, path))
# print(path)

Shortest Path Length:  4975.982
Shortest Path Elevation Change:  54
Maximum Theoretical Length:  7463.973
[69025899, 69071341, 69071905, 69057871, 69069408, 69011898, 69038537, 69071341, 69071905, 69057871, 69069408, 69011898, 69044182, 68970727, 69029257, 68980206, 69048114, 68997244, 68963642, 5916268622, 69069740, 68980659, 69048969, 68961526, 69028126, 68995324, 69028571, 68982213, 69033033, 68958587, 68959400, 69053365, 69000615, 69005434, 1937532207, 65811502, 65854123, 65875610, 65777405]
Maximum Elevation Path Length:  7405.0999999999985
Maximum Elevation Path Elevation Change: 63


In [61]:
dijkstra = StrategyDijkstra(graph)
path = dijkstra.get_route(69025899,65777405)
print(graph_utils.get_path_length(graph, path))

4975.982
